```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = None
```

# 1. 데이터 탐색

## 1) 데이터 불러오기, info, describe 확인, 결측치 확인
```python
train = pd.read_csv('train.csv')
train.head(2)

test = pd.read_csv('test.csv')
test.head(2)

train.info()
test.info()

train.describe()
test.describe()

train.isna().sum() / len(train)
test.isna().sum() / len(test)
```

## 2) 종속변수(타겟변수) 확인
```python
train['종속변수명'].value_counts()
```

## 3) 숫자형 변수와 범주형 변수 분리
```python
numeric_list = []
categorical_list = []

for i in train.columns:
    if train[i].dtypes == 'O':
        categorical_list.append(i)
    else:
        numeric_list.append(i)
       
# 종속변수가 들어있는 리스트에서 종속변수 제거
종속변수들어간리스트_list.remove('종속변수 컬럼명')

# 숫자형 변수와 종속변수의 상관성 확인
for i in numeric_list:
    answer = train[[i, '종속변수 컬럼명']].corr()
    print(answer)
    print()
```

## 4) 독립변수와 종속변수의 관계 확인
```python
train.columns
print(train.groupby(['독립변수 컬럼명'])['종속변수 컬럼명'].mean())     # 분류문제 = mean(), 예측문제 = sum()
```

## 5) 컬럼의 nunique, unique 확인
```python
column_list = train.columns

for i in column_list:
    answer = train[i].nunique()
    print(i, answer)
    
for i in column_list:
    answer = train[i].unique()
    print(i, answer)
```

## 6) 컬럼별 시각화
```python
# 분포 확인 시각화
num_list = ['분포 확인할 컬럼명']

for i in num_list:
    sns.displot(train[i])
    
# 범주별 종속변수 시각화
cat_list = ['범주형 컬럼명']

for i in cat_list:
    sns.catplot(data = train, x = i, hue = '종속변수 컬럼명', kind = 'count')
```

# 2. 전처리

## 1) datetime(날짜컬럼) 전처리
```python
train['datetime'] = pd.to_datetime(train['datetime'])

train['year'] = train['datetime'].dt.year
train['month'] = train['datetime'].dt.month
train['day'] = train['datetime'].dt.day
train['hour'] = train['datetime'].dt.hour
train['dayofweek'] = train['datetime'].dt.dayofweek

print(train['추가한 컬럼명'].unique())
print(train.groupby(['추가한 컬럼명'])['종속변수 컬럼명'].mean())     # 또는 sum()

train = train.drop(columns = ['필요없는 컬럼명'])

# 최종적으로 추가한 컬럼 테스트 데이터에도 동일하게 적용(train, test 컬럼 동일해야 함)
test['datetime'] = pd.to_datetime(test['datetime'])

test['year'] = test['datetime'].dt.year
test['month'] = test['datetime'].dt.month
test['day'] = test['datetime'].dt.day
test['hour'] = test['datetime'].dt.hour
test['dayofweek'] = test['datetime'].dt.dayofweek
```

## 2) 불필요한 컬럼 삭제
```python
y = train['종속변수 컬럼명']
train = train.drop(columns = ['삭제할 컬럼명'])

submission_id = test['id']     # 필요없을 수도 있음
test = test.drop(columns = ['삭제할 컬럼명'])
```

## 3) 결측치 처리
```python
train.isna().sum() / len(train)
test.isna().sum() / len(test)

# 처리방법1. 특정값으로 대체(median, mean, value_counts의 가장 많은 값)
train['결측치 존재 컬럼명'] = train['결측치 존재 컬럼명'].fillna(대체할 값)
test['결측치 존재 컬럼명'] = test['결측치 존재 컬럼명'].fillna(대체할 값)

# 처리방법2. 결측치 있는 컬럼 삭제
train = train.drop(columns = ['결측치 존재 컬럼명'])
test = test.drop(columns = ['결측치 존재 컬럼명'])
```

## 4) 범주형 변수 인코딩
```python
# 처리방법1. one hot encoding
train = pd.get_dummies(train, columns = ['범주형 컬럼명'], drop_first = True)
test = pd.get_dummies(test, columns = ['범주형 컬럼명'], drop_first = True)

# 처리방법2. Label encoding(회귀 관련 알고리즘에는 사용 어려움. 단, tree계열은 괜찮음)
from sklearn.preprocessing import LabelEncoder

le_list = ['범주형 컬럼명']

for i in le_list:
    encoder = LabelEncoder()
    train[i] = encoder.fit_transform(train[i])
    test[i] = encoder.fit_transform(test[i])
```

## 5) 파생변수 추가
- train, test 모두에 추가해야함. train, test 데이터의 컬럼 동일해야 함

## 6) 표준화(스케일링)
```python
# 처리방법1. StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train = pd.DataFrame(scaler.fit_transform(train), columns = train.columns)
test = pd.DataFrame(scaler.fit_transform(test), columns = test.columns)

# 처리방법2. MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

train = pd.DataFrame(scaler.fit_transform(train), columns = train.columns)
test = pd.DataFrame(scaler.fit_transform(test), columns = test.columns)
```

## 7) 컬럼 간 상관관계 확인
```python
train[['컬럼명1', '컬럼명2', ...]].corr()

# 0.6이상인 컬럼들이 있으면 둘 중 한 컬럼 삭제
train = train.drop(columns = ['삭제할 컬럼명'])
test = test.drop(columns = ['삭제할 컬럼명'])
```

# 3. 학습 및 평가

## 1) train 데이터를 학습용과 검증용으로 분할
```python
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train, y, test_size = 0.3, stratify = y, random_state = 2022)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
```

## 2) 학습 - 분류(성별 분류, 생존 여부 분류 등)의 경우
```python
# 모델1: RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state = 2022, max_depth = 5, n_estimators = 200)
model.fit(x_train, y_train)

# 모델2: XGBClassifier
from xgboost import XGBClassifier
model = XGBClassifier(random_state = 2022, max_depth = 5, n_estimators = 200)
model.fit(x_train, y_train)
```

## 2) 학습 - 예측(수요 예측 등)의 경우
```python
# 모델1: RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state = 2022, max_depth = 5, n_estimators = 200)
model.fit(x_train, y_train)

# 모델2: XGBRegressor
from xgboost import XGBRegressor
model = XGBRegressor(random_state = 2022, max_depth = 5, n_estimators = 200)
model.fit(x_train, y_train)
```

## 3) 모델 성능 평가 - 분류의 경우
```python
y_test_predicted = pd.DataFrame(model.predict(x_test))

from sklearn.metrics import roc_auc_score, accuracy_score
print(roc_auc_score(y_test, y_test_predicted))
print(accuracy_score(y_test, y_test_predicted))
```

## 3) 모델 성능 평가 - 예측의 경우
```python
y_test_predicted = pd.DataFrame(model.predict(x_test))

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
print(r2_score(y_test, y_test_predicted))
print(mean_squared_error(y_test, y_test_predicted))     # MSE
print(mean_squared_error(y_test, y_test_predicted, squared = False))     # RMSE
print(mean_absolute_error(y_test, y_test_predicted))     # MAE
```

## 4) 하이퍼파라미터 조절
```python
from sklearn.model_selection import GridSearchCV

params = {'n_estimators' : [100, 200, 300, 400, 500], 'max_depth' : [5, 7, 9, 11, 13]}

model_hy = RandomForestClassifier(random_state = 2022)
grid_cv = GridSearchCV(model_hy, param_grid = params, cv = 3, scoring = '모델 성능 평가 이름')
# scoring 예: 'roc_auc', 'accuracy', 'r2', 'neg_mean_squared_error', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'
grid_cv.fit(x_train, y_train)

print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
```

## 5) 모델 재학습 및 성능평가
- 위에서 나온 최적 하이퍼 파라미터를 적용해 2) 학습 및 3) 모델 성능 평가 재수행

## 6) 중요 변수 파악
```python
pd.Series(model.feature_importances_, index = x_train.columns).sort_values(ascending = False)
```

## 7) 최종 결과 예측
```python
# predict_proba: 분류에서 0인지 1인지가 아니라 0일 확률, 1일 확률을 구하는 경우
result = pd.DataFrame(model.predict_proba(test))[0 또는 1]     # 0일 확률이면 0, 1일 확률이면 1

# predict: 분류에서 0인지 1인지 구하는 경우 또는 예측 문제의 경우
result = pd.DataFrame(model.predict(test))
```

# 4. 결과 제출

```python
final = pd.concat([submission_id, result], axis = 1)     # 최종 데이터에 id컬럼 추가해야 한다면! 아니면 result만 출력
final = final.rename(columns = {'변경 필요한 컬럼명' : '변경할 컬럼명'})
final.to_csv('파일명.csv', index = False)
```